In [ ]:

import itertools
from tqdm import tqdm
from collections import Counter
# from arrs import *
import math
import numpy as np
from sympy import primefactors, sieve
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import networkx as nx


from itertools import chain, combinations, combinations_with_replacement

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


def bin_operation(a, b, mod):
    return (a*b)%mod



def check_idemp(a, mod):
    return bin_operation(a, a, mod) == a



def absorption_laws(a, b, mod1, mod2):
    return bin_operation(a, bin_operation(a, b, mod1), mod2 ) == a and bin_operation(a, bin_operation(a, b, mod2), mod1 ) == a



def distributivity(a, b, c, mod1, mod2):
    return bin_operation(a, bin_operation(b, c, mod2), mod1) == bin_operation( bin_operation(a, b, mod1), bin_operation(a, c, mod1), mod2)

def is_bounded(s, mod1):
    for can_z in s:
        c = 1
        for j in s:
            if bin_operation(j, can_z, mod1) != j:
                c = 0
                break
        if c == 1:
            return can_z
    return '-'


def is_distributiv(setp, mod1, mod2):
    for i in combinations_with_replacement(setp, 3):
        if distributivity(*i, mod1, mod2) == 0:
            return 0
    return 1

def check_ordered(setp, mod):
    for i in combinations(setp, 2):
        if bin_operation(i[0], i[1], mod) not in i:
            return 0
    return 1

def check_ordered_double(setp, mod1, mod2):
    for i in combinations(setp, 2):

        if bin_operation(i[0], i[1], mod1) not in i:
            return 0

        if bin_operation(i[0], i[1], mod2) not in i:
            return 0

        if bin_operation(i[0], i[1], mod1) ==  bin_operation(i[0], i[1], mod2) :
            #print(i[0], i[1], mod1, mod2)
            return 0

    return 1



inters = lambda a, b: tuple(set(a) & set(b))

def is_a_semigroup(setp, mod):
    for i in combinations_with_replacement(setp, 2):
        if bin_operation(i[0], i[1], mod) not in setp:
            return 0
    return 1


def is_a_lattice(setp, mod1, mod2):
    for c in combinations_with_replacement(setp, 2):
        if absorption_laws(c[0], c[1], mod1, mod2) == 0:
            return 0
    return 1

def make_a_table(setp, mod):
    d = {setp[i]: i for i in range(len(setp))}

    l = len(setp)

    M = np.zeros((l, l))

    for i in range(l):
        for j in range(l):
            M[i, j] = d[bin_operation(setp[i], setp[j], mod)]

    return M

def isomorphism_two_table(M1, M2):

    if M1.shape != M2.shape:
        return 0

    if  (np.array_equal(M1, M2) or
         np.array_equal(np.transpose(M1), M2) or
         np.array_equal(M1, np.transpose(M2)) or
         np.array_equal(np.transpose(M1), np.transpose(M2)) ):

        return 1

def check_isomorphism(setp, mod1, mod2, p = 0):
    M1 = make_a_table(setp, mod1)
    M2 = make_a_table(setp, mod2)
    if p == 1:
        print(M1)
        print(M2)

    return isomorphism_two_table(M1, M2)


def characterise(setp, mod1, mod2):
    d1 = is_distributiv(setp, mod1, mod2)
    d2 = is_distributiv(setp, mod2, mod1)

    d4 = check_ordered(setp, mod1)
    d5 = check_ordered(setp, mod2)

    d7 = check_ordered_double(setp, mod1, mod2)

    d6 = check_isomorphism(setp, mod1, mod2)

    b1 = is_bounded(setp, mod1)
    b2 = is_bounded(setp, mod2)

    print( "bounded: ", b1, b2, "ordered 2: ", d7 ,'is distributiv ', d1, d2,' is  ordered ', d4, d5, ' is isom ', d6,  setp, mod1, mod2)



def make_a_matrix(setp, mod):
    l = len(setp)
    M = np.zeros((l, l))

    for i in range(l):
        for j in range(l):
            M[i, j] = ((setp[i]*setp[j])%mod == setp[i])

    return M



def find_one_nums_like_h():
    arr = []
    for i in tqdm(range(2, 2_000)):
        out = []
        for j in range(0, i):
            if check_idemp(j, i):
                out.append(j)
        t = 0

        r = (out, i)

        if len(out) > 3:
            arr.append(r)

    out = []

    for k in tqdm(combinations(arr, 2)):
        t = inters(k[0][0], k[1][0])
        if len(t) > 3:
            g = (sorted(t),  k[0][1], k[1][1] ) 
            out.append(g)
            #print(g)

    for p in out:

        for i in powerset(p[0]):
            if len(i) == 3 and i[0] != 0:
                if is_a_semigroup(i, p[1]) and is_a_semigroup(i, p[2]):
                    if is_a_lattice(i,  p[1],  p[2]):

                        characterise(i,  p[1],  p[2])



def find_sets_like_hilbert():
     for i in range(1, 31):
        t = 0

        #print(i)
        for s in powerset(range(0, i)):
            c = 1
            

            for k in combinations_with_replacement(s, 2):
                if k[0] == k[1]:
                    if (k[0] * k[0])%i != k[0]:
                        c = 0 
                        break

                if (k[0] * k[1])%i not in s:
                    c = 0 
                    break

            if c == 1 and s != () :
                #print([s, i], ',')
                t +=1

        print(i, t)


import matplotlib.pyplot as plt
import networkx as nx

def show_graph_with_labels(adjacency_matrix, mylabels):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500, labels=mylabels, with_labels=True)
    plt.show()

def visualize(arr, mod):
    M = make_a_matrix(arr, mod)
    print(M)
    mylabels = {i:str(arr[i]) for i in range(len(arr))}
    G = nx.from_numpy_matrix(M, create_using=nx.DiGraph)
    options = {
    'node_color': 'white',
    'node_size': 400,
    'width': 1,
    'arrowstyle': '-|>',
    'arrowsize': 22,
    }
    nx.draw_circular(G, labels=mylabels, arrows=True, **options)
    plt.draw()
    plt.show()

if __name__ == "__main__":
    #find_sets_like_hilbert()
    find_one_nums_like_h()
    #(0, 190, 855, 1065), 1330, 1995

    # visualize((0, 385, 595, 925), 1155)
    # visualize((0, 385, 595, 925), 2310)

    #check_isomorphism((0, 190, 855, 1065) ,1330, 1995, 1)

100%|██████████| 1998/1998 [00:00<00:00, 3252.77it/s]
1385280it [00:02, 582347.62it/s]


bounded:  - 441 ordered 2:  0 is distributiv  1 1  is  ordered  0 1  is isom  None (231, 385, 441) 462 770
bounded:  - 666 ordered 2:  0 is distributiv  1 1  is  ordered  0 1  is isom  None (190, 456, 666) 798 1330
bounded:  400 666 ordered 2:  0 is distributiv  1 1  is  ordered  1 1  is isom  None (210, 400, 666) 798 1330
bounded:  1065 - ordered 2:  0 is distributiv  1 1  is  ordered  1 0  is isom  None (190, 855, 1065) 1330 1995
bounded:  1065 400 ordered 2:  0 is distributiv  1 1  is  ordered  1 1  is isom  None (210, 400, 1065) 1330 1995


In [ ]:
({666, 1036, 1296, 1666}, * mod 2590, * mod 3330) is a bounded lattice lmao

In [ ]:
from itertools import product 
from tqdm import tqdm 
out = []
for s in tqdm (product(range(1, 20), repeat = 2)):
    terms = list(s)
    c = 0
    for i in range(10000):

        terms.append(int(str(terms[-1]+terms[-2])[::-1]))
        if terms[-1] < 10:
            c += 1
        terms = terms[1:]
    out.append(c)
    #print(s, c)
print(out)


In [ ]:
terms = [9, 9]
for i in range(10000):

    terms.append(int(str(terms[-1]+terms[-2])[::-1]))
print(terms)

[9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81, 9, 9, 81,

In [ ]:
from itertools import chain, combinations, combinations_with_replacement, permutations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(2, len(s)+1))

def permutations_of_powerset(iterable):
    p = powerset(iterable)
    out = []
    for i in p:
        for j in permutations(i):
            out.append(j)
    return out

def unicue_permutations_of_powerset(iterable):
    p = permutations_of_powerset(iterable)
    out = []
    while len(p) > 0:
        i = p[-1]
        out.append(i)
        p = [j for j in p if ''.join(map(str, j)) not in ''.join(map(str, i))]
    
    return out


s = [1,2,3,4,5,6]
print(set(unicue_permutations_of_powerset(s)) == set(permutations(s))  )


True


In [ ]:
from itertools import product
for R in range(100):
    c = 0
    for s in product(range(2*R + 1), repeat = 4):
        if (2*R - 1)**2 <= 4*sum((i - R)**2 for i in s) <= (2*R + 1)**2: c += 1
    print(c if R != 0 else 1, end = ', ')

1, 32, 200, 528, 1280, 2744, 4272, 6592, 10144, 15048, 19824, 25824, 34744, 43520, 55184, 64680, 80864, 99184, 115616, 135144, 157344, 185872, 207304, 239600, 272960, 310240, 351096, 385392, 433040, 485528, 531728, 583696, 646056, 714800, 779488, 842928, 921176, 1001136, 1086000, 1163064, 1263760, 1372736, 1463536, 1563816, 1682096, 1809488, 1903064, 2048464, 2183360, 2330752, 2485032, 2602192, 2774128, 2953592, 3114080, 3257424, 3470968, 3674976, 3844768, 4041600, 4270920, 4487728, 4712032, 4911320, 5171696, 5451056, 5674672, 5929352, 6199872, 6525104, 

KeyboardInterrupt: ignored